In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 30
verbosity = 2
input_shape = (width, height, channel)

In [3]:
# ResNet50 모델 사용

pretrained_model = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
)

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

pretrained_model.trainable=True
set_trainable=False

for layer in pretrained_model.layers:
    if layer.name == 'block7c_project_conv':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False


model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 2603 images belonging to 2 classes.
Found 386 images belonging to 2 classes.


In [5]:
es_callback = EarlyStopping(monitor='val_auc', mode='max', patience=8,
                                              verbose=1, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//batch_size,
                    callbacks= [es_callback],
                    verbose=verbosity,
                   )

Epoch 1/30
40/40 - 28s - loss: 0.5968 - auc: 0.7453 - accuracy: 0.6755 - val_loss: 0.3991 - val_auc: 0.8926 - val_accuracy: 0.8359
Epoch 2/30
40/40 - 26s - loss: 0.4081 - auc: 0.9070 - accuracy: 0.8106 - val_loss: 0.3088 - val_auc: 0.9384 - val_accuracy: 0.8880
Epoch 3/30
40/40 - 27s - loss: 0.3273 - auc: 0.9560 - accuracy: 0.8834 - val_loss: 0.2871 - val_auc: 0.9668 - val_accuracy: 0.9036
Epoch 4/30
40/40 - 27s - loss: 0.2822 - auc: 0.9717 - accuracy: 0.9043 - val_loss: 0.2512 - val_auc: 0.9809 - val_accuracy: 0.9245
Epoch 5/30
40/40 - 26s - loss: 0.2500 - auc: 0.9808 - accuracy: 0.9236 - val_loss: 0.2032 - val_auc: 0.9891 - val_accuracy: 0.9427
Epoch 6/30
40/40 - 27s - loss: 0.2170 - auc: 0.9887 - accuracy: 0.9449 - val_loss: 0.1998 - val_auc: 0.9936 - val_accuracy: 0.9453
Epoch 7/30
40/40 - 27s - loss: 0.1918 - auc: 0.9924 - accuracy: 0.9508 - val_loss: 0.1847 - val_auc: 0.9955 - val_accuracy: 0.9557
Epoch 8/30
40/40 - 27s - loss: 0.1750 - auc: 0.9955 - accuracy: 0.9634 - val_loss: 

In [6]:
prediction = model.predict(valid_generator)

In [7]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

315
71


In [8]:
prediction

array([[0.99118084],
       [0.9507828 ],
       [0.99938786],
       [0.99989116],
       [0.98057276],
       [0.04668803],
       [0.92314535],
       [0.10500804],
       [0.99711347],
       [0.1901638 ],
       [0.99999356],
       [0.9907403 ],
       [0.941668  ],
       [0.4987978 ],
       [0.99896324],
       [0.9990822 ],
       [0.01041231],
       [0.9579229 ],
       [0.9449037 ],
       [0.9405934 ],
       [0.99979407],
       [0.997336  ],
       [0.9989862 ],
       [0.9847164 ],
       [0.99466264],
       [0.02588991],
       [0.06692401],
       [0.992312  ],
       [0.81446993],
       [0.99798137],
       [0.9371875 ],
       [0.13987227],
       [0.94224477],
       [0.66230434],
       [0.00923813],
       [0.9295495 ],
       [0.9703169 ],
       [0.96820575],
       [0.94109106],
       [0.40357918],
       [0.9989297 ],
       [0.74424714],
       [0.9062585 ],
       [0.9158702 ],
       [0.9145449 ],
       [0.99997973],
       [0.5293972 ],
       [0.999

In [9]:
tf.__version__

'2.3.1'

In [10]:
model.evaluate(valid_generator)

7/7 [==============================] - 3s 420ms/step - loss: 0.1082 - auc: 1.0000 - accuracy: 0.9870


[0.10815528780221939, 0.9999526739120483, 0.9870466589927673]